In [14]:
import numpy as np
import random
import re

# oh so pretty!
def float_formatter(x):
    if x < 0.001 and x > 0:
        return "   >0" 
    elif x == 0:
        return "    0" 
    else:
        return "%.3f" % x
np.set_printoptions(formatter={'float_kind':float_formatter})

def _logsum(logx, logy):
    print("Test")
    print(logx)
    print(logy)
    if len(logx.shape) < 1:
        if np.isinf(logx) or np.isinf(logy):
            print("result")
            print(np.max([logx, logy]))
            return np.max([logx, logy])
        else:
            print("result")
            print(np.logaddexp(logx, logy))
            return np.logaddexp(logx, logy)
    else:
        b = np.any([np.isinf(logx), np.isinf(logy)], axis=1)
        print(b)
        result = np.zeros(shape=logx.shape[0])
        result[b] = np.maximum(logx[b], logy[b])
        result[not b] = np.logaddexp(logx[not b], logy[not b])
        print("result")
        print(result)
        return result

def write_fasta_file(annotation_file_to_save, annotation_string, genome_file=None):
    if (genome_file!=None):
        with open(genome_file) as gf:
            first_line = gf.readline()
            s = " "
            words = first_line.split(s)
            words[0] = words[0] + " gene annotation"
            first_line = s.join(words[:-1])
    else:
        first_line = "; gene annotation"
    with open(annotation_file_to_save, 'w+') as af:
        af.write(first_line + '\n')
        name = annotation_file_to_save.replace('/', '\\').split('\\')[-1].split('.fa')[0]
        af.write('>' + name + '\n')
        line_length = 60
        full_lines = len(annotation_string)//line_length
        for anot_line in range(full_lines):
            af.write(annotation_string[anot_line*line_length:(anot_line+1)*line_length] + '\n')
        af.write(annotation_string[(full_lines)*line_length:])
       
    
# eg
#write_fasta_file("data/annotation13.fa", "ABABABABABABABABABAAAAJHASJHASJJSdskfjklsfkjklj sdhjkøfhjkø ksdhjf jkjkhsdfjk hjksdf hjksdfjk jhksd fhjkdsfhjk AJJSJHASHJASJJHASJHASJHASJHSAJJHASJHASHJASJJHASJHASJJHASJH")

In [15]:
def read_fasta_file(filename):
    """
    Reads the given FASTA file f and returns a dictionary of sequences.

    Lines starting with ';' in the FASTA file are ignored.
    """
    sequences_lines = {}
    current_sequence_lines = None
    with open(filename) as fp:
        for line in fp:
            line = line.strip()
            if line.startswith(';') or not line:
                continue
            if line.startswith('>'):
                sequence_name = line.lstrip('>')
                current_sequence_lines = []
                sequences_lines[sequence_name] = current_sequence_lines
            else:
                if current_sequence_lines is not None:
                    current_sequence_lines.append(line)
    sequences = {}
    for name, lines in sequences_lines.items():
        sequences[name] = ''.join(lines)
    return sequences

def load_data(n):
    train_genome = ''
    train_annotation = ''
    indices = [1, 2, 3, 4, 5]
    test_genome = ''
    test_annotation = ''
    for i in indices:
        if i != n:
            train_genome += read_fasta_file('data/genome%d.fa' % i)['genome%d' % i]
            train_annotation += read_fasta_file('data/annotation%d.fa' % i)['annotation%d' % i]
        else:
            test_genome += read_fasta_file('data/genome%d.fa' % i)['genome%d' % i]
            test_annotation += read_fasta_file('data/annotation%d.fa' % i)['annotation%d' % i]
    return train_genome, train_annotation, test_genome, test_annotation


In [16]:
#First we create our model:
    states = ['noncoding', 'c-start-codon', 'c-content-codon', 'c-end-codon', 'r-start-codon', 'r-content-codon', 'r-end-codon']
    
    # define all possible outcomes of all states:
    outcomes_dict = {} # ohh gawd this is akward
    outcomes = np.empty([68], dtype=object) # help me, im lost
    i = 0
    letters = 'ACGT'
    for letter1 in letters:
        outcomes[i] = letter1
        outcomes_dict[letter1] = i
        i += 1
        for letter2 in letters:
            for letter3 in letters:
                outcomes[i] = letter1 + letter2 + letter3
                outcomes_dict[letter1 + letter2 + letter3] = i
                i += 1

def build_model(train_genome, train_annotation):
    #First we create our model:
    states = ['noncoding', 'c-start-codon', 'c-content-codon', 'c-end-codon', 'r-start-codon', 'r-content-codon', 'r-end-codon']
    
    # define all possible outcomes of all states:
    outcomes_dict = {} # ohh gawd this is akward
    outcomes = np.empty([68], dtype=object) # help me, im lost
    i = 0
    letters = 'ACGT'
    for letter1 in letters:
        outcomes[i] = letter1
        outcomes_dict[letter1] = i
        i += 1
        for letter2 in letters:
            for letter3 in letters:
                outcomes[i] = letter1 + letter2 + letter3
                outcomes_dict[letter1 + letter2 + letter3] = i
                i += 1
    
    A = np.zeros((len(states), len(states)))
    E = np.zeros((len(states), outcomes.shape[0]))
    pi = np.zeros((len(states)))
    pi[0] = 1
    #Now we have to fill out the data.
    #First we take care of transitions
    A[0, 1] = train_annotation.count('NC')#N -> C
    A[3, 0] = train_annotation.count('CN')# C -> N
    A[0, 4] = train_annotation.count('NR')#N -> R 
    A[6, 0] = train_annotation.count('RN')#R -> N
    A[3, 4] = train_annotation.count('CR')#C -> R
    A[6, 1] = train_annotation.count('RC') #R -> C
    A[0, 0] = train_annotation.count('N') - (A[0,1] + A[0,4])
    A[1, 2] = 1 # C-start -> C-content
    A[4, 5] = 1 # R-stop -> R-content
    A[2, 2] = train_annotation.count('CCC') - (A[0, 1] + A[3,0] + A[3,4] + A[6,1]) # c-content -> c-content
    A[5, 5] = train_annotation.count('RRR') - (A[0, 4] + A[6,0] + A[6,1] + A[3,4]) # r-content -> r-content
    A[2, 3] = A[3, 4] + A[3, 0]
    A[5, 6] = A[6, 1] + A[6, 0]
    A = A/A.sum(axis=1)[:, None]
    #Now we need to find emissions.
    for m in re.finditer('CCC', train_annotation):
        index = m.start()
        genome_string = outcomes_dict[train_genome[index:index+3]]
        if train_annotation[index-1] == 'C':
            if train_annotation[index+3] == 'C':
                E[2, genome_string] += 1
            else:
                E[3, genome_string] += 1
        else:
            E[1, genome_string] += 1
    for m in re.finditer('RRR', train_annotation):
        index = m.start()
        genome_string = outcomes_dict[train_genome[index:index+3]]
        if train_annotation[index-1] == 'R':
            if train_annotation[index+3] == 'R':
                E[5, genome_string] += 1
            else:
                E[6, genome_string] += 1
        else:
            E[4, genome_string] += 1
    
    for m in re.finditer('N', train_annotation):
        index = m.start()
        E[0, outcomes_dict[train_genome[index]]] += 1
    
    E = E/E.sum(axis=1)[:, None]
    return A, E, pi

IndentationError: unexpected indent (<ipython-input-16-454a907aab0d>, line 2)

In [17]:
def l2i(l):
    if type(l) is str:
        if l == "" or len(l) == 2:
            return -1
        else:
            return outcomes_dict[l]
    else:
        return [l2i(l_) for l_ in l]
#i can either be a list of indices, or just an index
def i2l(i):
    if type(i) is int:
        return outcomes[i]
    else:
        return [outcomes[i_] for i_ in i]
def e_prop(E, s):
    result = list()
    for i in range(len(s)):
        result.append(E[i, s[i]])
    return result

In [18]:
#This is the Viterbi algorithm, where different states, can take input of different length.
#A, E, p and s is as above. q is a vector, signifying the length of inputs for each state.

s = test_genome

def logaddexplist(v1, v2):
    result = np.logaddexp(v1, v2) 
    b1 = np.isinf(v1)
    b2 = np.isinf(v2)
    result[b1] = v2[b1]
    result[b2] = v1[b2]
    return result


def viterbi_ext_gen(A, E, p, s):
    A = np.log(A)
    E = np.log(E)
    p = np.log(p)
    #Then we generate some result matrix, with -inf everywhere.
    result = np.log(np.zeros([A.shape[0], len(s)]))
    
    #First we make the base cases
    obs = [s[i] for i in range(3)]
    #first row only relies on the probabolity of starting in a state and the emission probability of the first observation.
    result[:, 0] = p + E[:, l2i(obs[0])]
    #second row relies on the probability of coming from the first state, and the emission probability of the second observation
    result[:, 1] = (result[:, 0] + (A[:, :].T)).max(axis=1) + E[:, l2i(obs[1])]
    
    # The third row relies on the probability of starting in a state, and emission of the first three observations.
    # After that it relies on the chance of coming from either an earlier state, and the last emission, or the first state,
    # two emissions (not necesary in our case.)
    result[:, 2] = np.maximum(p + E[:, l2i(obs[0] + obs[1] + obs[2])],
                                (result[:, 1] + (A[:, :].T)).max(axis=1) + E[:, l2i(obs[2])])
    
    for i in range(3, len(s)):
        obs = [s[i-j] for j in range(2, -1, -1)]
        result[:, i] = np.maximum((result[:, i-3] + (A[:, :].T)).max(axis=1) + E[:, l2i(obs[0] + obs[1] + obs[2])],
                                (result[:, i-1] + (A[:, :].T)).max(axis=1) + E[:, l2i(obs[2])])
    print(result)
    return result

def viterbi_ext_path(X, A, E, p, s):
    A = np.log(A)
    E = np.log(E)
    p = np.log(p)
    path = -1*np.ones(X.shape[1], dtype=np.int32)
    n = X.shape[1]-1
    path[n] = X[:, n].argmax()
    n -= 1
    while n >= 2:
        obs = [s[n-1], s[n], s[n+1]]
        temp = np.zeros((X.shape[0], 2))
        temp[:, 0] = (X[:, n-2] + A[:, path[n+1]]) + E[path[n+1], l2i(obs[0] + obs[1] + obs[2])]
        temp[:, 1] = (X[:, n] + A[:, path[n+1]]) + E[path[n+1], l2i(obs[2])]
        state, step = np.unravel_index(temp.argmax(), temp.shape)
        if step == 0:
            path[n-2] = state
            n -= 3
        else:
            path[n] = state
            n -= 1
    while n >= 0:
        path[n] = 0
        n -= 1
    return path[path > -1]

def viterbi_ext_annotation(genome, path):
    result = np.empty(len(path), dtype=object)
    result[path == 0] = 'N'
    result[path == 1] = 'CCC'
    result[path == 2] = 'CCC'
    result[path == 3] = 'CCC'
    result[path == 4] = 'RRR'
    result[path == 5] = 'RRR'
    result[path == 6] = 'RRR'
    result = "".join(result)
    return result
#print(A)
#X = viterbi_ext_gen(A, E, pi, s)
#print("boom")
#print(X[:, -3:])

#path = viterbi_ext_path(X, A, E, pi, s)
#print(path)
#print((path!=0).sum())
#for i in range(path.shape[0]):
#    if path[i] == 1:
#        print(X[:, i-2:i+2])
#        break
#annotation_pred = viterbi_ext_annotation(s, path)
#print(len(test_genome))
#print(len(annotation_pred))
#print(annotation_pred)
#write_fasta_file('data/annotation_prediction_5.fa', annotation_pred, 'data/genome5.fa')

In [11]:
S = np.arange(4).reshape(2,2)
S = S.T
S[0,1] = 4
print(S)
print(S[1, :])
print(S.argmax())

[[0 4]
 [1 3]]
[1 3]
1


In [19]:
#
# compare_anns.py <true> <pred>
#
# compares a predicted gene structure against the true gene structure and computes
# various statistics summarizing the quality of the prediction. The argument <true> is 
# the true gene structure in faste format, and <pred> is the predicted gene structure 
# in fasta format, e.g.
#
# > python compare_anns.py ./annotation1.fa ./pred1.fa
# > Only Cs (tp=728238, fp=0, tn=505177, fn=249):
# > Sn = 0.9997, Sp = 1.0000, CC = 0.9996, AC = 0.9996
# > Only Rs (tp=618777, fp=0, tn=505426, fn=0):
# > Sn = 1.0000, Sp = 1.0000, CC = 1.0000, AC = 1.0000
# > Both (tp=1347015, fp=0, tn=505177, fn=249):
# > Sn = 0.9998, Sp = 1.0000, CC = 0.9997, AC = 0.9997
#
# Christian Storm <cstorm@birc.au.dk>
#

import os
import sys
import string
import math

def read_ann(filename):
    lines = []
    for l in open(filename).readlines():
        if l[0] != ">" and l[0] != ';':
            lines.append(l.strip())
    return "".join(lines)

def count_c(true, pred):
    total = tp = fp = tn = fn = 0
    for i in range(len(true)):
        if pred[i] == 'C' or pred[i] == 'c':
            total = total + 1
            if true[i] == 'C' or true[i] == 'c':
                tp = tp + 1
            else:
                fp = fp + 1
        if pred[i] == 'N' or pred[i] == 'n':
            if true[i] == 'N' or true[i] == 'n' or true[i] == 'R' or true[i] == 'r':
                tn = tn + 1
            else:
                fn = fn + 1
    return(total, tp, fp, tn, fn)

def count_r(true, pred):
    total = tp = fp = tn = fn = 0
    for i in range(len(true)):
        if pred[i] == 'R' or pred[i] == 'r':
            total = total + 1
            if true[i] == 'R' or true[i] == 'r':
                tp = tp + 1
            else:
                fp = fp + 1
        if pred[i] == 'N' or pred[i] == 'n':
            if true[i] == 'N' or true[i] == 'n' or true[i] == 'C' or true[i] == 'c':
                tn = tn + 1
            else:
                fn = fn + 1
    return(total, tp, fp, tn, fn)

def count_cr(true, pred):
    total = tp = fp = tn = fn = 0
    for i in range(len(true)):
        if pred[i] == 'C' or pred[i] == 'c' or pred[i] == 'R' or pred[i] == 'r':
            total = total + 1
            if (pred[i] == 'C' or pred[i] == 'c') and (true[i] == 'C' or true[i] == 'c'):
                tp = tp + 1
            elif (pred[i] == 'R' or pred[i] == 'r') and (true[i] == 'R' or true[i] == 'r'):
                tp = tp + 1                
            else:
                fp = fp + 1
        if pred[i] == 'N' or pred[i] == 'n':
            if true[i] == 'N' or true[i] == 'n':
                tn = tn + 1
            else:
                fn = fn + 1
    return(total, tp, fp, tn, fn)

def print_stats(tp, fp, tn, fn):
    sn = float(tp) / (tp + fn)
    sp = float(tp) / (tp + fp)
    cc = float((tp*tn - fp*fn)) / math.sqrt(float((tp+fn)*(tn+fp)*(tp+fp)*(tn+fn)))
    acp = 0.25 * (float(tp)/(tp+fn) + float(tp)/(tp+fp) + float(tn)/(tn+fp) + float(tn)/(tn+fn))
    ac = (acp - 0.5) * 2
    print("Sn = %.4f, Sp = %.4f, CC = %.4f, AC = %.4f" % (sn, sp, cc, ac))

def print_all(true, pred):
    (totalc, tp, fp, tn, fn) = count_c(true, pred)
    if totalc > 0:
        print("Only Cs (tp=%d, fp=%d, tn=%d, fn=%d):" % (tp, fp, tn, fn))
        print_stats(tp, fp, tn, fn)

    (totalr, tp, fp, tn, fn) = count_r(true, pred)
    if totalr > 0:
        print("Only Rs (tp=%d, fp=%d, tn=%d, fn=%d):" % (tp, fp, tn, fn))
        print_stats(tp, fp, tn, fn)

    (total, tp, fp, tn, fn) = count_cr(true, pred)
    if totalc > 0 and totalr > 0:
        print("Both (tp=%d, fp=%d, tn=%d, fn=%d):" % (tp, fp, tn, fn))
        print_stats(tp, fp, tn, fn)


# Read true annotation
true_ann = read_ann('data/annotation5.fa')

# Read predicted annotations
pred_ann = read_ann('data/annotation_prediction_5.fa')
# Check annoation length
error = 0
if len(true_ann) != len(pred_ann):
    print("ERROR: The lengths of two predictions are different")
    print("Expected %d, but found %d" % (len(true_ann), len(pred_ann)))
    sys.exit(1)    
    
# Print stats
print_all(true_ann, pred_ann)


Only Cs (tp=1067915, fp=109048, tn=399015, fn=39531):
Sn = 0.9643, Sp = 0.9073, CC = 0.7827, AC = 0.7834
Only Rs (tp=909552, fp=159954, tn=413627, fn=24919):
Sn = 0.9733, Sp = 0.8504, CC = 0.7424, AC = 0.7440
Both (tp=1977467, fp=269002, tn=374096, fn=64450):
Sn = 0.9684, Sp = 0.8803, CC = 0.6352, AC = 0.6417


In [20]:
for i in [1, 2, 3, 4, 5]:
    train_genome, train_annotation, test_genome, test_annotation = load_data(i)
    A, E, pi = build_model(train_genome, train_annotation)
    X = viterbi_ext_gen(A, E, pi, test_genome)
    path = viterbi_ext_path(X, A, E, pi, test_genome)
    pred_annotation = viterbi_ext_annotation(test_genome, path)
    write_fasta_file('data/annotation_prediction_'+ str(i) + '.fa', pred_annotation, 'data/genome'+ str(i) + '.fa')
    print_all(test_annotation, pred_annotation)

NameError: name 'build_model' is not defined